# SimHash实验结果分析

本notebook对SimHash算法在不同程度文本变化下的表现进行深入分析。通过分析汉明距离和相似度的变化模式，来探究SimHash算法的鲁棒性和敏感度特性。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import pearsonr

## 1. 实验数据

In [ ]:
# 实验数据
change_percentages = [10, 20, 30, 40, 50, 60, 70, 80, 90]
feature_counts = [50756, 45116, 39477, 33837, 28198, 22558, 16919, 11279, 5640]
hamming_distances = [0, 1, 0, 3, 5, 2, 4, 4, 6]
similarities = [1.0000, 0.9844, 1.0000, 0.9531, 0.9219, 0.9688, 0.9375, 0.9375, 0.9062]

# 创建数据框
data = pd.DataFrame({
    '文本变化比例(%)': change_percentages,
    '特征数量': feature_counts,
    '汉明距离': hamming_distances,
    '相似度': similarities
})

print(data)

## 2. 数据可视化

In [ ]:
# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 显示负号

# 创建2x2子图
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. 汉明距离随文本变化比例的变化
axes[0, 0].plot(change_percentages, hamming_distances, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_title('汉明距离随文本变化比例的变化', fontsize=15)
axes[0, 0].set_xlabel('文本变化比例(%)', fontsize=12)
axes[0, 0].set_ylabel('汉明距离', fontsize=12)
axes[0, 0].grid(True, linestyle='--', alpha=0.7)
axes[0, 0].set_xticks(change_percentages)

# 2. 相似度随文本变化比例的变化
axes[0, 1].plot(change_percentages, similarities, 'go-', linewidth=2, markersize=8)
axes[0, 1].set_title('相似度随文本变化比例的变化', fontsize=15)
axes[0, 1].set_xlabel('文本变化比例(%)', fontsize=12)
axes[0, 1].set_ylabel('相似度', fontsize=12)
axes[0, 1].grid(True, linestyle='--', alpha=0.7)
axes[0, 1].set_xticks(change_percentages)
axes[0, 1].set_ylim(0.85, 1.01)  # 设置y轴范围，更清晰地显示变化

# 3. 特征数量随文本变化比例的变化
axes[1, 0].plot(change_percentages, feature_counts, 'ro-', linewidth=2, markersize=8)
axes[1, 0].set_title('特征数量随文本变化比例的变化', fontsize=15)
axes[1, 0].set_xlabel('文本变化比例(%)', fontsize=12)
axes[1, 0].set_ylabel('特征数量', fontsize=12)
axes[1, 0].grid(True, linestyle='--', alpha=0.7)
axes[1, 0].set_xticks(change_percentages)

# 4. 汉明距离与特征数量的关系
axes[1, 1].scatter(feature_counts, hamming_distances, color='purple', s=100)
axes[1, 1].set_title('汉明距离与特征数量的关系', fontsize=15)
axes[1, 1].set_xlabel('特征数量', fontsize=12)
axes[1, 1].set_ylabel('汉明距离', fontsize=12)
axes[1, 1].grid(True, linestyle='--', alpha=0.7)

# 添加趋势线
z = np.polyfit(feature_counts, hamming_distances, 1)
p = np.poly1d(z)
axes[1, 1].plot(feature_counts, p(feature_counts), "r--", alpha=0.8)

plt.tight_layout()
plt.show()

## 3. 数据分析

In [ ]:
# 基本统计分析
print("汉明距离的基本统计量:")
print("最小值:", np.min(hamming_distances))
print("最大值:", np.max(hamming_distances))
print("平均值:", np.mean(hamming_distances))
print("中位数:", np.median(hamming_distances))
print("标准差:", np.std(hamming_distances))

print("\n相似度的基本统计量:")
print("最小值:", np.min(similarities))
print("最大值:", np.max(similarities))
print("平均值:", np.mean(similarities))
print("中位数:", np.median(similarities))
print("标准差:", np.std(similarities))

# 相关性分析
corr_change_hamming, p_change_hamming = pearsonr(change_percentages, hamming_distances)
corr_change_sim, p_change_sim = pearsonr(change_percentages, similarities)
corr_feature_hamming, p_feature_hamming = pearsonr(feature_counts, hamming_distances)

print("\n相关性分析:")
print(f"文本变化比例与汉明距离的相关系数: {corr_change_hamming:.4f}, p值: {p_change_hamming:.4f}")
print(f"文本变化比例与相似度的相关系数: {corr_change_sim:.4f}, p值: {p_change_sim:.4f}")
print(f"特征数量与汉明距离的相关系数: {corr_feature_hamming:.4f}, p值: {p_feature_hamming:.4f}")

## 4. 非线性模式分析

汉明距离随文本变化不是严格线性的，尝试进一步分析这种非线性模式。

In [ ]:
# 拟合多项式模型
degrees = [1, 2, 3]  # 尝试不同度的多项式

plt.figure(figsize=(14, 8))

plt.subplot(1, 2, 1)
plt.scatter(change_percentages, hamming_distances, color='blue', s=80, label='实际数据')

for degree in degrees:
    coeffs = np.polyfit(change_percentages, hamming_distances, degree)
    poly = np.poly1d(coeffs)
    
    # 生成更平滑的曲线
    x_smooth = np.linspace(min(change_percentages), max(change_percentages), 100)
    y_smooth = poly(x_smooth)
    
    plt.plot(x_smooth, y_smooth, label=f'{degree}次多项式拟合')

plt.title('汉明距离的多项式拟合模型', fontsize=14)
plt.xlabel('文本变化比例(%)', fontsize=12)
plt.ylabel('汉明距离', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()

# 计算每个文本变化比例对应的汉明距离增量
hamming_increments = [hamming_distances[i] - hamming_distances[i-1] if i > 0 else 0 for i in range(len(hamming_distances))]

plt.subplot(1, 2, 2)
plt.bar(change_percentages, hamming_increments, color='orange', alpha=0.7)
plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.title('汉明距离的增量变化', fontsize=14)
plt.xlabel('文本变化比例(%)', fontsize=12)
plt.ylabel('汉明距离增量', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 5. 敏感度阈值分析

探索在什么文本变化比例下，SimHash的稳定性开始显著下降。

In [ ]:
# 设置敏感度阈值
similarity_thresholds = [0.95, 0.93, 0.90]
hamming_thresholds = [3, 4, 6]

# 创建图表
plt.figure(figsize=(14, 6))

# 相似度阈值分析
plt.subplot(1, 2, 1)
plt.plot(change_percentages, similarities, 'go-', linewidth=2, markersize=8, label='相似度')

for threshold in similarity_thresholds:
    plt.axhline(y=threshold, color='r', linestyle='--', alpha=0.5, label=f'阈值: {threshold}')

plt.title('相似度阈值分析', fontsize=14)
plt.xlabel('文本变化比例(%)', fontsize=12)
plt.ylabel('相似度', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()
plt.ylim(0.85, 1.01)

# 汉明距离阈值分析
plt.subplot(1, 2, 2)
plt.plot(change_percentages, hamming_distances, 'bo-', linewidth=2, markersize=8, label='汉明距离')

for threshold in hamming_thresholds:
    plt.axhline(y=threshold, color='r', linestyle='--', alpha=0.5, label=f'阈值: {threshold}')

plt.title('汉明距离阈值分析', fontsize=14)
plt.xlabel('文本变化比例(%)', fontsize=12)
plt.ylabel('汉明距离', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend()

plt.tight_layout()
plt.show()

## 6. 总结与分析

基于以上实验结果，我们可以得出以下关于SimHash算法的性能特点与分析：

1. **稳定性与鲁棒性**
   - 即使文本变化率高达90%，SimHash的相似度仍然保持在0.9以上，汉明距离最大仅为6（相对于64位哈希值）
   - 在文本变化10%和30%的情况下，SimHash值完全相同（汉明距离为0），展示了算法对小规模内容变化的高度容忍性

2. **非线性敏感度模式**
   - 汉明距离随文本变化比例的增长呈现非线性模式，存在波动
   - 特别是在60%文本变化时出现"谷值"（汉明距离从5降至2），表明SimHash不仅与变化的数量有关，还与变化的内容特性相关

3. **阈值建议**
   - 根据实验结果，对于64位的SimHash，选择汉明距离阈值3-4是合理的
   - 这一阈值可接受40%-80%的文本变化，同时保持相似度在0.93以上

4. **应用价值**
   - SimHash在文档查重、网页去重和相似内容检测方面表现优异
   - 即使删除了90%的特征，相似度仍高达0.9062，证明算法对特征子集具有良好的代表性

5. **特殊观察**
   - 文本变化比例在30%处出现"奇异点"（汉明距离为0），这可能是由随机删除的特征集恰好保留了对哈希值影响最大的关键特征
   - 这一现象也证明了SimHash中某些特征对最终哈希值的贡献不平等，存在"关键特征"

这些发现对于调整SimHash参数、理解算法行为以及在实际应用中设置合理的相似度阈值具有重要指导意义。